<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Package-import" data-toc-modified-id="Package-import-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Package import</a></span></li><li><span><a href="#Data-Loading" data-toc-modified-id="Data-Loading-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Data Loading</a></span></li><li><span><a href="#Preprocessing" data-toc-modified-id="Preprocessing-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Preprocessing</a></span><ul class="toc-item"><li><span><a href="#parsing" data-toc-modified-id="parsing-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>parsing</a></span><ul class="toc-item"><li><span><a href="#typo_parser" data-toc-modified-id="typo_parser-3.1.1"><span class="toc-item-num">3.1.1&nbsp;&nbsp;</span>typo_parser</a></span></li><li><span><a href="#email_address_parser" data-toc-modified-id="email_address_parser-3.1.2"><span class="toc-item-num">3.1.2&nbsp;&nbsp;</span>email_address_parser</a></span></li><li><span><a href="#bytedata_parser" data-toc-modified-id="bytedata_parser-3.1.3"><span class="toc-item-num">3.1.3&nbsp;&nbsp;</span>bytedata_parser</a></span></li><li><span><a href="#structure_parser" data-toc-modified-id="structure_parser-3.1.4"><span class="toc-item-num">3.1.4&nbsp;&nbsp;</span>structure_parser</a></span></li><li><span><a href="#reference_parser" data-toc-modified-id="reference_parser-3.1.5"><span class="toc-item-num">3.1.5&nbsp;&nbsp;</span>reference_parser</a></span></li></ul></li><li><span><a href="#main-structural_email" data-toc-modified-id="main-structural_email-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>main structural_email</a></span><ul class="toc-item"><li><span><a href="#merged-features" data-toc-modified-id="merged-features-3.2.1"><span class="toc-item-num">3.2.1&nbsp;&nbsp;</span>merged features</a></span></li></ul></li><li><span><a href="#attribute-selection" data-toc-modified-id="attribute-selection-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>attribute selection</a></span><ul class="toc-item"><li><span><a href="#module-test" data-toc-modified-id="module-test-3.3.1"><span class="toc-item-num">3.3.1&nbsp;&nbsp;</span>module test</a></span></li></ul></li></ul></li></ul></div>

## Package import 

In [1]:
import pandas as pd 
import numpy as np
import os
from glob import glob
from tqdm import tqdm
import matplotlib.pyplot as plt
import mailparser
import re

## Data Loading
- from file into DataFrame

In [2]:
def load_data_folder(path):
    """
    @param folders: the train or test directory
    @return: document list with [doc_path, doc, label, original_idx]
    """
    folders = glob(path+"/**")  # explore all the folder under the directory

    docs = []
    for classes in folders:
        label = classes.split("\\")[-1]
        doc_paths = glob(classes+"\\**")
        
        for doc_path in doc_paths:
            original_idx = doc_path.split("\\")[-1]
            
            with open(doc_path, encoding="UTF-8") as f:
                text = f.read()
            docs.append([doc_path, text, label, original_idx])

    print(f"\nLoaded folder under {path}: \n")
    for folder in folders:
        print(folder)
        
    return docs


corpus_train_docs = load_data_folder(path="../data/train")
corpus_test_docs = load_data_folder(path="../data/test")


Loaded folder under ../data/train: 

../data/train\alt.atheism
../data/train\comp.graphics
../data/train\comp.os.ms-windows.misc
../data/train\comp.sys.ibm.pc.hardware
../data/train\comp.sys.mac.hardware
../data/train\comp.windows.x
../data/train\misc.forsale
../data/train\rec.autos
../data/train\rec.motorcycles
../data/train\rec.sport.baseball
../data/train\rec.sport.hockey
../data/train\sci.crypt
../data/train\sci.electronics
../data/train\sci.med
../data/train\sci.space
../data/train\soc.religion.christian
../data/train\talk.politics.guns
../data/train\talk.politics.mideast
../data/train\talk.politics.misc
../data/train\talk.religion.misc

Loaded folder under ../data/test: 

../data/test\alt.atheism
../data/test\comp.graphics
../data/test\comp.os.ms-windows.misc
../data/test\comp.sys.ibm.pc.hardware
../data/test\comp.sys.mac.hardware
../data/test\comp.windows.x
../data/test\misc.forsale
../data/test\rec.autos
../data/test\rec.motorcycles
../data/test\rec.sport.baseball
../data/test

## Preprocessing

### parsing

In [3]:
corpus_train = pd.DataFrame(corpus_train_docs, columns=["doc_path", "text", "label", "original_idx"])
corpus_train = corpus_train.reset_index().rename(columns={"index":"global_index"})

corpus_test = pd.DataFrame(corpus_test_docs, columns=["doc_path", "text", "label", "original_idx"])
corpus_test = corpus_test.reset_index().rename(columns={"index":"global_index"})

print("original_idx duplicate count:", corpus_train.shape[0] - corpus_train.original_idx.drop_duplicates().shape[0], " on ", corpus_train.shape[0])
print("original_idx duplicate count:", corpus_test.shape[0] - corpus_test.original_idx.drop_duplicates().shape[0], " on ", corpus_test.shape[0])

original_idx duplicate count: 1060  on  11083
original_idx duplicate count: 770  on  7761


#### typo_parser

In [476]:
def typo_parser(x):
    """
    1. replace irrelevant symbol "|" or "*"
    2. remove extra space "  "
    3. replace extra \n "\n\n" into "\n"
    4. replace "> *>" into ">>" for further analysis

    @param string: email body string
    @return: cleaned email body string, extracted emails
    
    # test_string = 'www.\n com\n\n or ?\n>\n    >>\n    \n > > >|> (note) \n> \n I\nam not good enough with regex>'
    # typo_parser(test_string)

    """
    # x = re.sub('([,:;?!\.”\)])\n', '\g<1> ', x)  # add space for symbol like .\n or ?\n
    # x = re.sub('(\w)\n(\w)', '\g<1> \g<2>', x)  # add space for symbol like word\nword
    x = re.sub('\n', ' \n ', x)  # add space for between \n
    x = re.sub("[\*|\|\^]", "", x) # replace irrelevant symbol "|" or "*"
    
    x = re.sub(">[ >]*>", ">>", x)# compress > [?] > 
    x = re.sub("\[.*?\]", "", x, flags=re.S)  # separate for typo like [a)
    x = re.sub("\(.*?\)", "", x, flags=re.S)

    x = re.sub("\n[ \n]*\n", "\n", x) # compress \n
    return x



#### email_address_parser

In [463]:
def email_address_parser(string):
    """
    extract and remove email from the body
    @param string: email body string
    @return: cleaned email body string, extracted emails
    """
    emails = None
    emails = re.findall(" ?[\S]+@[\S]+ ?", string)
    string = re.sub(" ?[\S]+@[\S]+ ?", " ", string)
    return string, emails

#### bytedata_parser

In [359]:
def bytedata_parser(string, threshold=50):
    """
    Since 99% of english words length ranged from [1,20], but consider special symbol there, we set the threshold with 50 for only parse bytdata like photo
    If length of span larger than threshold, then we will not treat it as a word. 
    sep can only use space
    """
    bytedata = None
    clean_string = " ".join([word for word in re.split(" ", string) if len(word)<=threshold])
    ## sentence length is the same
    # clean_string = "\n".join([word for word in re.split("\n", clean_string) if len(word)<=threshold])
    bytedata = [word for word in re.split(" ", string) if len(word)>threshold]
    return clean_string, bytedata

#### structure_parser

In [360]:
def structure_parser(string):
    """
    @param parser: email string
    @return: structural information for email header, body, others
    """
    error_message = None
    header = {}
    body = ""
    others = []
    try:
        mail = mailparser.parse_from_string(string)
        if mail.has_defects:  # [first line error]
            remove_first_line_string = "\n".join(string.split("\n")[1:])
            mail = mailparser.parse_from_string(remove_first_line_string)
            # print("remove_first_line_string update for ")
        header, body = mail.headers, mail.body
        others = [mail.date, mail.delivered_to, mail.to_domains, error_message]

    except Exception as error:
        error_message = error
    return header, body, others

#### reference_parser

In [485]:
def extra_parser(x):
    """
    remove_flag and extra space
    """
    x = re.sub("(?:In article)?.*writes:" , "", x, flags=re.S)
    x = re.sub(" {2,}", " ", x) # compress space
    return x

def reference_parser(string, match_type=2):
    """
    Consider reply with referencing previous email, we need to separate them to make prediction separately.
    @param 
        string: email body string
        match_type: 0 with return only main body, 1 with return main body + previous one reference, 2 with more reference
    @return: 
        reply, previous_one, previous_two in the email
    
    @ test with the following code
    string = " \n\n\n\n    >>>zero email \n\n >>first email\n >second email\n reply email \n"
    reply, previous_one, previous_two = reference_parser(string, match_type=2)
    print("## reply\n", repr(reply))
    print("## previous_one\n", repr(previous_one))
    print("## previous_two\n", repr(previous_two))
    """
    
    previous_one, previous_two, reply = '', '', ''

    # extract reply with out containing >
    reply = " ".join([s for s in string.split("\n") if ">" not in s])
    reply = extra_parser(reply)
    
    # add "\n" before string to matchign [^>]{1}
    if match_type>0:
        previous_one = " ".join(re.findall("[^>]{1}>{1}([^>]{1}[\S ]*)\n", "\n" + string)) # matching >
        previous_one = extra_parser(previous_one)
        
    if match_type>1: # flag reference_two
        previous_two = " ".join(re.findall("[^>]{1}>{2}([^>]{1}[\S ]*)\n", "\n" + string)) # matching >>
        previous_two = extra_parser(previous_two)
    # previous_two_more_pt = "[^>]{1}>{2,}([^>]{1}[\S ]*)\n" # matching >> or >>> more
    return reply, previous_one, previous_two

### main structural_email 

In [487]:

def structural_email(data, bytedata_parser_threshold=50, reference_parser_match_type=2):
    """
    This is a parser pipeline, parser order matters.
    1. string => structure email to separate => header, body, others
    2. body => remove typo and some irrelevant words => body
    3. body => parse and remove email from body => body_no_email
    4. body_no_email => parse and remove binary data like BMP or picture from body => body_no_binary_no_email
    5. body_no_binary_no_email => separate email reference and reply => reply, previous_one, previous_two
    
    @param data: data text dataframe series including all the training set or test set
    @return: structural information
    """
    print("Preprocessing for unstructure email...")
    header_info = []
    body_info = []
    others_info = []
    for string in tqdm(data):
        header, body, others = structure_parser(string)
        body = typo_parser(body)
        body_no_email, emails = email_address_parser(body)
        body_no_binary_no_email, bytedata = bytedata_parser(body_no_email, threshold=bytedata_parser_threshold)
        reply, previous_one, previous_two = reference_parser(body_no_binary_no_email, match_type=reference_parser_match_type)

        header_info.append(header)
        body_info.append([reply, previous_one, previous_two])
        others_info.append(others+[emails]+[bytedata])

    a1 = pd.DataFrame.from_dict(header_info)
    a2 = pd.DataFrame(body_info, columns=["reply", "reference_one", "reference_two"])
    a3 = pd.DataFrame(others_info, columns=["date", "delivered_to", "to_domains", "error_message", "contained_emails", "long_string"])
    structure_email = pd.concat([a1, a2, a3], axis=1)
    return structure_email

#### merged features 

In [488]:
structural_train = structural_email(corpus_train["text"])
structural_test = structural_email(corpus_test["text"])

  0%|                                                                                | 1/11083 [00:00<25:29,  7.25it/s]

Preprocessing for unstructure email...


 54%|████████████████████████████████████████▊                                   | 5957/11083 [00:55<00:44, 114.32it/s]Email content 'x-usenet-faq' not handled
Email content 'x-usenet-faq' not handled
Email content 'x-usenet-faq' not handled
  0%|▎                                                                              | 27/7761 [00:00<00:28, 267.29it/s]

Preprocessing for unstructure email...


100%|██████████████████████████████████████████████████████████████████████████████| 7761/7761 [01:33<00:00, 83.25it/s]


In [519]:
train = pd.concat([corpus_train, structural_train], axis=1)
test = pd.concat([corpus_test, structural_test], axis=1)
all_cols = train.columns.tolist()
print(all_cols)

['global_index', 'doc_path', 'text', 'label', 'original_idx', 'From', 'Subject', 'Summary', 'Keywords', 'Expires', 'Distribution', 'Organization', 'Supersedes', 'Lines', 'X-Newsreader', 'NNTP-Posting-Host', 'Reply-To', 'Nntp-Posting-Host', 'In-Reply-To', 'News-Software', 'X-Mailer', 'Originator', 'Article-I.D.', 'X-News-Reader', 'X-Sender', 'X-Disclaimer', 'Nntp-Posting-User', 'X-Bytes', 'X-Xxmessage-Id', 'X-Xxdate', 'X-Useragent', 'In-reply-to', 'OD-Comment-To', 'ReplyTo', 'Disclaimer', 'Comments', 'Posting-Front-End', 'X-Reader', 'Mime-Version', 'Content-Type', 'Content-Transfer-Encoding', 'X-UserAgent', 'X-NewsSoftware', 'Nntp-Software', 'Oganization', 'Apparently-To', 'X-Comment-To', 'X-Gateway', 'X-Advert', 'Cc', 'X-News-Software', 'X-Posted-From', 'Follow-Ups', 'X-Auth-User', 'X-FTN-To', 'X-Gated-By', 'X-Standard-Disclaimer', 'Moderator', 'X-XXMessage-ID', 'X-XXDate', 'To', 'Posted-Date', 'Received-Date', 'Orginization', 'X-Md4-Signature', 'Return-Receipt-To', 'X-Mail-Reader', 'C

### attribute selection

In [520]:
t = train.isnull().sum().sort_values()
not_used_cols = t[t > train.shape[0]*0.1].index.tolist()
print("not_used_cols: \n", not_used_cols)

select_cols = ["global_index", "doc_path", "label", "reply", "reference_one", "reference_two",
               "Subject", "From", "Lines", "Organization", "contained_emails", "long_string", "text", "error_message"]
print("may use cols: \n", select_cols)
t.to_frame().style

not_used_cols: 
 ['Distribution', 'Nntp-Posting-Host', 'NNTP-Posting-Host', 'Reply-To', 'Keywords', 'Article-I.D.', 'X-Newsreader', 'Summary', 'Originator', 'In-Reply-To', 'News-Software', 'Expires', 'In-reply-to', 'To', 'X-Disclaimer', 'Disclaimer', 'X-Mailer', 'X-Useragent', 'X-Xxdate', 'Supersedes', 'X-UserAgent', 'X-XXDate', 'Nf-From', 'Nf-ID', 'X-News-Reader', 'X-Added', 'Original-Sender', 'X-Posted-From', 'X-XXMessage-ID', 'Content-Type', 'X-Xxmessage-Id', 'Cc', 'Mime-Version', 'X-Last-Updated', 'Moderator', 'Content-Transfer-Encoding', 'date', 'X-To', 'X-Gated-By', 'X-Sender', 'X-Received', 'Return-Path', 'Nntp-Posting-User', 'X-Auth-User', 'X-X-From', 'X-Md4-Signature', 'X-Maildoor', 'X-NewsSoftware', 'MIME-Version', 'Return-Receipt-To', 'X-AltNet-ID', 'X-Cc', 'Bcc', 'X-Software', 'Posting-Front-End', 'Comments', 'X-Newssoftware', 'X-News-Software', 'X-Reader', 'X-Bytes', 'X-Copyright', 'X-Mail-Reader', 'X-Header', 'Apparently-To', 'Organisation', 'Mmdf-Warning', 'OD-Comment-To

,0
global_index,0
doc_path,0
text,0
label,0
original_idx,0
delivered_to,0
reference_two,0
reference_one,0
reply,0
to_domains,11


In [521]:
train = train[select_cols]
test = test[select_cols]

In [522]:
train[["global_index", "reply", "reference_one", "reference_two",]].sample(3, random_state=1160).style

,global_index,reply,reference_one,reference_two
699,699,"A 68070 is just a 68010 with a built in MMU. I don't even think that Moto. manufactures them. - Ian Romanick Dancing Fool of Epsilon Were the contained thoughts 'opinions', EPN.NTSC.quality = Best PSU would probably not agree with them. ""Look, I don't know anything about douche, but I do know Anti-Freeze when I see it!"" - The Dead Milkmen","Wow! A 68070! I'd be very interested to get my hands on one of these, especially considering the fact that Motorola has not yet released the 68060, which is supposedly the next in the 680x0 lineup. 8-D",
4339,4339,They were designed for speeds of upwards of 80 - I forget the exact spec - but for military vehicles. That's 80 in a 1958 Dodge Powerwagon. Not 80 in a 1993 Ford Taurus.,I guess I wasn't clear enough here. I said the roads WERE designed for speeds of 80 or so and still be safe. The current 55-65 will add a saftey margin.,
9187,9187,"There is a veto proof majority in the house. The Senate, unfortunately, is a different story. The Lt.Gov. has vowed that the bill will not be voted on, and he has the power to do it. In addition, the Senate is a much smaller, and more readily manipulated body. On ther other hand, the semi-automatic ban will likely not live, as at least fifty per cent of the house currently opposes it, and it is VERY far down in the bill order in the Senate . And I thought my TX Political Science class was a waste of time! -- Dillon Pyron The opinions expressed are those of the TI/DSEG Lewisville VAX Support sender unless otherwise stated. 462-3556 492-4656 God gave us weather so we wouldn't complain about other things. PADI DM-54909",Do people expect the Texans congressmen to act as the N.J. Republicans did?,"Wasn't she the one making the comment in '88 about George being born with a silver foot in his mouth? Sounds like another damn politician to me. Ain't like the old days in Texas anymore. The politicians may have been corrupt then, but at least they'd take a stand. News now is that the House may already have a two-thirds majority, so her ""opposition"" out of her concern for image may not matter."


#### module test

In [501]:
def checking_text(idx, write_in_local=True):
    x = train[train["global_index"] == idx]
    string = x["text"].iloc[0]
    body = x["reply"].iloc[0]
    x_path = x["doc_path"].iloc[0]
    x_label = x["label"].iloc[0]
    
    if write_in_local:
        with open("E:/wyang_github/Text-Classification/text.txt", "w", encoding="utf-8") as f:
            f.write(x_label+"\n\n")
            f.write(x_path+"\n\n")
            f.write(string)
    return string, body, x_path, x_label


module_test = True

if module_test:
    # 可以分开一个pyfile, 并且把这里的过程保存下来, 然后写在report中
    # idx = 22
    idx = 9187

    string, reply, x_path, x_label = checking_text(idx)

    header, body, others = structure_parser(string)
    print("\nrepr(header):   \n", repr(header))
    print("\nrepr(body):   \n", repr(body))
    print("\nrepr(others):   \n", repr(others))

    body = typo_parser(body)
    print("\nrepr(body):   \n", repr(body))

    body_no_email, emails = email_address_parser(body)
    print("\nrepr(body):   \n", repr(body))
    print("\nrepr(emails):   \n", repr(emails))
    print("\nrepr(body_no_email):   \n", repr(body_no_email))

    body_no_binary_no_email, bytedata = bytedata_parser(body_no_email, threshold=25)
    print("\nrepr(bytedata):   \n", repr(bytedata))
    print("\nrepr(body_no_binary_no_email):   \n", repr(body_no_binary_no_email))

    reply, previous_one, previous_two = reference_parser(body_no_binary_no_email, match_type=2)

    print("\nrepr(reply):   \n", repr(reply))
    print("\nrepr(previous_one):   \n", repr(previous_one))
    print("\nrepr(previous_two):   \n", repr(previous_two))


repr(header):   
 {'From': 'pyron@skndiv.dseg.ti.com (Dillon Pyron)', 'Subject': 'Re: Founding Father questions', 'Lines': '35', 'Nntp-Posting-Host': 'skndiv.dseg.ti.com', 'Reply-To': 'pyron@skndiv.dseg.ti.com', 'Organization': 'TI/DSEG VAX Support'}

repr(body):   
 '\nIn article <1993Apr5.153951.25005@eagle.lerc.nasa.gov>, pspod@bigbird.lerc.nasa.gov (Steve Podleski) writes:\n>arc@cco.caltech.edu (Aaron Ray Clements) writes:\n>>Wasn\'t she the one making the comment in \'88 about George being born with\n>>a silver foot in his mouth?  Sounds like another damn politician to me.\n>>\n>>Ain\'t like the old days in Texas anymore.  The politicians may have been\n>>corrupt then, but at least they\'d take a stand.  (My apologies to a few\n>>exceptions I can think of.)  \n>>\n>>News now is that the House may already have a two-thirds majority, so \n>>her "opposition" out of her concern for image (she\'s even said this\n>>publicly) may not matter.\n>\n>Do people expect the Texans congressmen 

In [527]:
l = train['long_string'].dropna().apply(len).sort_values()
longest_idx = l[l>0].index.tolist()[-10:]
train[train.global_index.isin(longest_idx)]
# train['long_string'].sample(10).tolist()

,global_index,doc_path,label,reply,reference_one,reference_two,Subject,From,Lines,Organization,contained_emails,long_string,text,error_message
1362,1362,../data/train\comp.os.ms-windows.misc\9704,comp.os.ms-windows.misc,"Hi, everybody: I guess my subject has said it...","P!%U]< %HUIU 'QFO0""/5DT .UV4 $$ Y SLD9&1D>SLBP...","G P---7+'U0/,PURM8MMVG<GX6%A85""45%147!14:U1<%%...",Re: More Cool BMP files??,james@dlss2 (James Cummings),1021,RedRock Development,"[ <1993Apr17.023017.17301@gmuvax2.gmu.edu> , r...",[B_:S]K'EYK/W]_?W]K/W]_?W]_?W]_?VL_6JLKPO+R\_O...,From: james@dlss2 (James Cummings)\nSubject: R...,None
1635,1635,../data/train\comp.os.ms-windows.misc\9980,comp.os.ms-windows.misc,------------ Part 5 of 14 ------------ M%6M%:...,W&JVM<G)RK:UR<K>W<JVM<G)R<AM<GWMWRMO MK16MMZTR...,,roman.bmp 05/14,pwiseman@salmon.usd.edu (Cliff),957,University of South Dakota,"[ M,P$HWA=@8!SKWI_,S""B<)S<W-R>!5C<-S`HFNAIR<K)...","[M)B9R&KJZ)EVZ=+ITNKH>?7U=PGU]+""Q=?72_=""PU'GT>...",Subject: roman.bmp 05/14\nFrom: pwiseman@salmo...,None
1637,1637,../data/train\comp.os.ms-windows.misc\9982,comp.os.ms-windows.misc,------------ Part 6 of 14 ------------ MAX=7H...,'AX>'AX>'AX>'AX>'AX>'AX>' GHJKN/=)Q GK!Q 7EY>...,,roman.bmp 06/14,pwiseman@salmon.usd.edu (Cliff),958,University of South Dakota,"[ M`0'UZY\WG@&>T)YU)3A&$+&]#+&NAX>'AX>'AS$Q,3%...","[MAZ)75U=7R!1KP""5A8=F>`7;UG)\&V0&>YZ>W5&$'Z]M;...",Subject: roman.bmp 06/14\nFrom: pwiseman@salmo...,None
1638,1638,../data/train\comp.os.ms-windows.misc\9983,comp.os.ms-windows.misc,"------------ Part 8 of 14 ------------ M:J""N6...",'AX>'AX>'AX>'AX>'AX>'AX>'`812E0<'GIYX`5Z> `0&$...,,roman.bmp 08/14,pwiseman@salmon.usd.edu (Cliff),956,University of South Dakota,"[ MAX>'AX>'AX>'AX>'AYZ>8=#0T""L!A%<-N;@+<-NPP25...","[MAPW/HFN`LC4U-34U,AK:VLV-MD'E94'V=DV-FMKS\];6...",Subject: roman.bmp 08/14\nFrom: pwiseman@salmo...,None
1639,1639,../data/train\comp.os.ms-windows.misc\9984,comp.os.ms-windows.misc,------------ Part 10 of 14 ------------ MV=E&...,"'AX>'AX>'AX>'AX=34U-34U-3,3$Q,3B8G) 'AX>'AX>'A...",,roman.bmp 10/14,pwiseman@salmon.usd.edu (Cliff),956,University of South Dakota,[ M:FIJ:FIJ:FJ@KHB24E)24G9V=G9V=G9V=G9V=G9VE)3...,"[M:J""U$$,UM/3T]F9F9`P,#7EY>NRIZWT='I0MFQ#&'AX>...",Subject: roman.bmp 10/14\nFrom: pwiseman@salmo...,None
1640,1640,../data/train\comp.os.ms-windows.misc\9985,comp.os.ms-windows.misc,"MFYLH`A;P#AU M9&1D9&ID1?U MFSXH$-#B""'3B8-C8V-...","GIZ>GIZBHJGIZ>M_?1T>EI0L+""V9F9F9FR,A_?W_0& NRH...",,roman.bmp 09/14------------ Part 9 of 14 -----...,pwiseman@salmon.usd.edu (Cliff),958,University of South Dakota,[ MC#`P8V-55555555559-65@HWGV`7]?7U%Q>?GY?GP!@...,[MAX>'AX>'AX>'AX>'AX>'AX>'AX>'AX>'AX>'AX>'AX>'...,Subject: roman.bmp 09/14------------ Part 9 of...,None
1641,1641,../data/train\comp.os.ms-windows.misc\9986,comp.os.ms-windows.misc,------------ Part 11 of 14 ------------ M9V=G...,"7EZ]'1Z6EI0L+9F9F9F9F9F;$Q,3$Q,3$Q,3$9F8+""PL+ ...",,roman.bmp 11/14,pwiseman@salmon.usd.edu (Cliff),956,University of South Dakota,"[ MAS'$Q,1F9@L+""Z6E1T??1T??WT>EI0L+""PMF""Z5'1T?...","[MR1865%22DM75U=75U4)""0IV=G9V=G9V=G9V=G9V=G9V=...",Subject: roman.bmp 11/14 \nFrom: pwiseman@salm...,None
1642,1642,../data/train\comp.os.ms-windows.misc\9987,comp.os.ms-windows.misc,------------ Part 13 of 14 ------------ MTMB8...,"BHJNTM)Q&1D9L+""PL+""PL+!34U-34U-3L+""P 'AX>'AX>'...",,roman.bmp 13/14,pwiseman@salmon.usd.edu (Cliff),956,University of South Dakota,[ MAX>'AX>'AX>'AX>'AX>'AX>'AX>'AX>'AS'$9F9F9@M...,[M9\4TBU3&7$]F0->GK?1Z4+9C&'AX>'AX>'AX>'AX>'AX...,Subject: roman.bmp 13/14\nFrom: pwiseman@salmo...,None
1643,1643,../data/train\comp.os.ms-windows.misc\9988,comp.os.ms-windows.misc,------------ Part 12 of 14 ------------ MN/3T...,"' 'AX>' 'AS'$""T=ZBHJKN_?>GIZ>GK?WT=' NRIZWT>E""...",,roman.bmp 12/14,pwiseman@salmon.usd.edu (Cliff),956,University of South Dakota,"[ MAX>',<3$9@L+I:5'W]_?W]_?W]_?>GIZBJ_?>M_?W]_...",[MAX>'AX>'AX>'AX>'AX>'AX>'AX>'AX>'AX>'AX>'AX>'...,Subject: roman.bmp 12/14 \nFrom: pwiseman@salm...,None
9625,9625,..

In [517]:
t = train[train["reply"].apply(lambda x: "In article" in x)]
# t.groupby("label").size()